# Russell Wilson MVP Voting Adversaries

T. Shafer, 2020-10-02  
https://tshafer.com/blog/2020/10/russell-wilson-mvp

I was listening to one of Bill Simmons’s podcasts, and Russell Wilson came up as a topic. Wilson is a superb NFL quarterback, but apparently he has never received a single MVP vote. Setting aside MVP winners, who _has_ received one or more votes over the last several years?

In [ ]:
!which python

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## Data

I've collected data from across the web.

In [ ]:
df = pd.read_csv("nfl-mvp-voting.csv")
df.head()

Add a few columns marking whether a player won the MVP in a year, has ever won, and the number of votes received over time.

In [ ]:
df.drop(columns=["source", "accessed"], inplace=True)

df["winner"] = df.groupby("year")["votes"].transform(lambda c: c == c.max()).astype(int)
df["ever_won"] = df.groupby("player")["winner"].transform(max)

# Cumulative and total votes over time
df["cmlvotes"] = df.sort_values(["year", "player"]).groupby("player")["votes"].cumsum()
df["totvotes"] = df.groupby("player")["cmlvotes"].transform(max)

# Cumulative and total votes over time for *nonwinning* years
df["cmlvotes_nonwin"]=df.sort_values(["year", "player"]).groupby("player", as_index=False).apply(lambda x: (x["votes"] * (1-x["winner"])).cumsum()).reset_index(level=0,drop=True)
df["totvotes_nonwin"]=df.groupby("player")["cmlvotes_nonwin"].transform(max)

df.head()

## Generate the bar chart

In [ ]:
GOLDEN_SECTION = (1 + np.sqrt(5)) / 2
FIGURE_HEIGHT = 6

In [ ]:
plt.rcParams["font.size"] = 16
plt.rcParams["figure.dpi"] = 72

In [ ]:
df = df.sort_values(["totvotes", "year"], ascending=[True, False])

In [ ]:
fig, ax = plt.subplots(figsize=(GOLDEN_SECTION * FIGURE_HEIGHT, FIGURE_HEIGHT))

ax.barh(
    df.player, 
    df.cmlvotes, 
    ec='w', 
    lw=2, 
    color=['#d1d0ca' if p.winner else 'b' for _, p in df.iterrows()], 
    height=0.9,
)

# Custom font: https://concoursefont.com
# [t.set_fontname("Concourse T3") for t in ax.get_xticklabels()]
# [t.set_fontname("Concourse T3") for t in ax.get_yticklabels()]

ax.set_ylim(-0.9, 16.9)
ax.set_xlim(0, 60)

ax.xaxis.grid(alpha=0.5, ls="--")
ax.set_axisbelow(True)

ax.xaxis.set_ticks_position("both")
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
ax.xaxis.set_tick_params(direction="in", which="major", size=6)
ax.xaxis.set_tick_params(direction="in", which="minor", size=3)

ax.set_xlabel("Total MVP Votes", fontdict={"fontname": "Concourse T3"})
ax.set_title("NFL MVP Votes, 2014–2019", fontdict={"fontname": "Concourse T3"})

fig.tight_layout()
plt.savefig("mvp-votes.png", dpi=plt.rcParams["figure.dpi"], facecolor="w", edgecolor="w")
plt.savefig("mvp-votes@2x.png", dpi=plt.rcParams["figure.dpi"] * 2, facecolor="w", edgecolor="w");

## Table of nonwinning vote counts

In [ ]:
player_nonwin_counts = (
    df.loc[df.winner==0]
      .groupby("player", as_index=False)
      .agg({"totvotes_nonwin": max, "year": len})
)

player_nonwin_counts = (
    df.loc[:, ["player", "ever_won"]]
      .drop_duplicates()
      .merge(player_nonwin_counts, on="player")
)

player_nonwin_counts.loc[:, ["totvotes_nonwin", "year"]] = (
    player_nonwin_counts.loc[:, ["totvotes_nonwin", "year"]].astype(int)
)

player_nonwin_counts.sort_values(["year", "totvotes_nonwin"], ascending=False)